In [1]:
import tensorflow as tf
from mtcnn import MTCNN
from pathlib import Path
import pandas as pd
import glob
import cv2
import csv
import os
import ast
import pydot
import pydotplus
import graphviz
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import time

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback

In [3]:
# ===================== MULTITASK MODEL SETUP =====================

base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Flatten Layer
flatten = tf.keras.layers.Flatten(name='flatten')(base_model.output)

# Embedding Layer
embedding = tf.keras.layers.Dense(256, activation='relu', name='embedding')(flatten)
dropout4 = tf.keras.layers.Dropout(0.2, name='dropout4')(embedding)

# Landmarks Layers
landmarksDense_1 = tf.keras.layers.Dense(64, activation='relu', name='landmarksDense_1')(dropout4)
landmarksDropout1 = tf.keras.layers.Dropout(0.2, name='landmarksDropout1')(landmarksDense_1)
landmarksDense_2 = tf.keras.layers.Dense(64, activation='relu', name='landmarksDense_2')(landmarksDropout1)
landmarksDropout2 = tf.keras.layers.Dropout(0.2, name='landmarksDropout2')(landmarksDense_2)
landmarks = tf.keras.layers.Dense(10, activation='relu', name='landmark_output')(landmarksDropout2)

# Illuminance Layers
illuminanceDense_1 = tf.keras.layers.Dense(64, activation='relu', name='illuminanceDense_1')(dropout4)
illuminanceDropout1 = tf.keras.layers.Dropout(0.2, name='illuminanceDropout1')(illuminanceDense_1)
illuminanceDense_2 = tf.keras.layers.Dense(64, activation='relu', name='illuminanceDense_2')(illuminanceDropout1)
illuminanceDropout2 = tf.keras.layers.Dropout(0.2, name='illuminanceDropout2')(illuminanceDense_2)
illum = tf.keras.layers.Dense(1, activation='relu', name='previous_illuminance_output')(illuminanceDropout2)

# Retinex Layers

# Reshape
reshape = tf.keras.layers.Reshape((4, 4, 16), name='reshape')(dropout4)

# Deconvolution Block 1
deconv1 = tf.keras.layers.Conv2DTranspose(128, (2, 2), activation='relu', name='deconv1')(reshape)
dropout1 = tf.keras.layers.Dropout(0.2, name='retinexDropout1')(deconv1)
upsample1 = tf.keras.layers.UpSampling2D((2, 2), name='upsample1')(dropout1)  # Adjust size

# Deconvolution Block 2
deconv2 = tf.keras.layers.Conv2DTranspose(64, (4, 4), activation='relu', name='deconv2')(upsample1)
dropout2 = tf.keras.layers.Dropout(0.2, name='retinexDropout2')(deconv2)
upsample2 = tf.keras.layers.UpSampling2D((2, 2), name='upsample2')(dropout2)  # Adjust size

# Deconvolution Block 3
deconv3 = tf.keras.layers.Conv2DTranspose(32, (7, 7), activation='relu', name='deconv3')(upsample2)
dropout3 = tf.keras.layers.Dropout(0.2, name='retinexDropout3')(deconv3)
upsample3 = tf.keras.layers.UpSampling2D((7, 7), name='upsample3')(dropout3)  # Adjust size

# Conv2D layer for final output
retIllum = tf.keras.layers.Conv2D(3, kernel_size=(3, 3), activation='relu', padding='same', name='image_retinex_output')(upsample3)

task_outputs = [landmarks, illum, retIllum]

multi_task_model = tf.keras.Model(inputs=base_model.input, outputs=task_outputs)

# Compile the model with specific loss functions and metrics for each task
multi_task_model.compile(
    optimizer=Adam(learning_rate=1e-6),
    loss={
        'landmark_output': 'mean_squared_error',
        'previous_illuminance_output': 'mean_squared_error',
        'image_retinex_output': 'mean_squared_error'
    },
    metrics={
        'landmark_output': ['mse', "mae"],
        'previous_illuminance_output': ['mse', "mae"],
        'image_retinex_output': ['mse', "mae"]
    }
)

# Summary of the multi-task model
multi_task_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

 landmarksDropout2 (Dropout)    (None, 64)           0           ['landmarksDense_2[0][0]']       
                                                                                                  
 illuminanceDropout2 (Dropout)  (None, 64)           0           ['illuminanceDense_2[0][0]']     
                                                                                                  
 upsample3 (UpSampling2D)       (None, 224, 224, 32  0           ['retinexDropout3[0][0]']        
                                )                                                                 
                                                                                                  
 landmark_output (Dense)        (None, 10)           650         ['landmarksDropout2[0][0]']      
                                                                                                  
 previous_illuminance_output (D  (None, 1)           65          ['illuminanceDropout2[0][0]']    
 ense)    